In [1]:
import matplotlib.pyplot as plt
import time
import random
import numpy as np
import os
import cv2
import copy
import pandas as pd
import warnings
from tqdm import tqdm
import pprint


#org_path= 'G:/Projects/Python/Evolution_Simulation/'
org_path='/mnt/Personal/Projects/Python/Evolution_Simulation/'

frames_path = org_path+'tax_frames'
video_path = org_path+'tax_plots.mp4'


In [2]:
def blend_colors(color1, color2, color3, frac1, frac2, frac3):
    """
    Blend three hex colors based on given fractions.

    Args:
    - color1, color2, color3: Hex color strings (e.g., "#FF0000" for red).
    - frac1, frac2, frac3: Fractions corresponding to each color.

    Returns:
    - Blended color as a hex string.
    """
    # Convert hex color to RGB tuple
    def hex_to_rgb(hex_color):
        hex_color = hex_color.lstrip('#')
        return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

    # Convert RGB tuple to hex color
    def rgb_to_hex(rgb_color):
        return '#{:02x}{:02x}{:02x}'.format(*rgb_color)

    # Get RGB values for each color
    rgb1 = hex_to_rgb(color1)
    rgb2 = hex_to_rgb(color2)
    rgb3 = hex_to_rgb(color3)

    # Calculate blended RGB values
    blended_rgb = (
        int(frac1 * rgb1[0] + frac2 * rgb2[0] + frac3 * rgb3[0]),
        int(frac1 * rgb1[1] + frac2 * rgb2[1] + frac3 * rgb3[1]),
        int(frac1 * rgb1[2] + frac2 * rgb2[2] + frac3 * rgb3[2])
    )

    # Convert blended RGB values back to hex
    blended_hex = rgb_to_hex(blended_rgb)
    return blended_hex

In [3]:
def invert_hex(hex_color):
    hex_color = hex_color.lstrip('#')
    rgb = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
    inverted_rgb = tuple(255 - value for value in rgb)
    inverted_hex = '#{:02x}{:02x}{:02x}'.format(*inverted_rgb)
    return inverted_hex

def hex_to_rgb(hex_colors):
    rgb_colors = []
    for hex_color in hex_colors:
        # Strip the '#' symbol if present
        hex_color = hex_color.lstrip('#')
        
        # Convert to RGB tuple
        rgb = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
        rgb_colors.append(rgb)
    
    return rgb_colors

def weighted_average_color(hex_colors, fractions):
    # Convert hex colors to RGB tuples
    rgb_colors = hex_to_rgb(hex_colors)
    
    # Calculate weighted average RGB value
    weighted_rgb = [int(sum(f * c[i] for f, c in zip(fractions, rgb_colors))) for i in range(3)]
    
    # Ensure RGB values are within range [0, 255]
    weighted_rgb = [max(0, min(255, val)) for val in weighted_rgb]
    
    # Convert weighted RGB tuple back to hex color
    weighted_hex = '#{:02X}{:02X}{:02X}'.format(*weighted_rgb)
    
    return weighted_hex

def rgb_to_hex(rgb):
    """Converts an RGB tuple to a hexadecimal color code."""
    return '#{:02x}{:02x}{:02x}'.format(rgb[0], rgb[1], rgb[2])

def generate_rgb_colors(n):
    # Define endpoints of RGB scale
    start_color = np.array([0, 0, 255])  # Blue (RGB: 0, 0, 255)
    end_color = np.array([255, 0, 0])   # Red (RGB: 255, 0, 0)

    # Generate equally spaced colors
    colors = []
    for i in range(n):
        # Linear interpolation between start_color and end_color
        color = tuple(np.round(start_color + (end_color - start_color) * (i / (n - 1))).astype(int))
        colors.append(color)

    return colors

def draw_ngon(data, labels,max_plots,decimal):

    values_list=data[-max_plots:]

    n=len(data[0])

    assert len(values_list) > 0, "At least one set of values must be provided"
    assert all(len(values) == n for values in values_list), f"All value sets must have length {n}"
    assert len(labels) == n, "The number of labels must be equal to n"

    # Generate the vertices of the regular n-gon
    angles = np.linspace(0, 2 * np.pi, n, endpoint=False)
    vertices = np.c_[np.cos(angles), np.sin(angles)]

    # Plot the n-gon
    fig, ax = plt.subplots(figsize=(5.2, 5.1))  # Adjust figure size as needed
    polygon = plt.Polygon(vertices, closed=True, fill=None, edgecolor=invert_hex('#777777'), linewidth=2.5)  # Red color for edges, thicker border
    ax.add_patch(polygon)

    # Initialize variables for drawing lines
    center = np.mean(vertices, axis=0)  # Center of the n-gon
    prev_point = None  # To track the previous point for drawing lines

    # Plot each set of values
    for values in values_list:
        # Calculate the coordinates of the point based on the values
        point = np.dot(values, vertices)

        # Draw straight lines from previous point to current point
        if prev_point is not None:
            ax.plot([prev_point[0], point[0]], [prev_point[1], point[1]], 'k-')

        # Update previous point
        prev_point = point

    colors = generate_rgb_colors(n)  # Define colors for partitions manually

    for i in range(len(colors)):
        colors[i]=rgb_to_hex(colors[i])
        colors[i]=invert_hex(colors[i])

    dot_color=weighted_average_color(colors,values_list[-1])
    #dot_color=invert_hex(dot_color)

    # Plot a smaller dot at the last point
    ax.plot(prev_point[0], prev_point[1], 'o', markersize=5,color=dot_color)

    # Annotate the vertices with labels
    label_offset = 0.05  # Offset factor for label positions
    for i, vertex in enumerate(vertices):
        alignment = 'right' if vertex[0] < 0 else 'left'
        vertical_alignment = 'top' if vertex[1] < 0 else 'bottom'
        offset_x = label_offset if vertex[0] >= 0 else -label_offset
        offset_y = label_offset if vertex[1] >= 0 else -label_offset
        ax.text(vertex[0] + offset_x, vertex[1] + offset_y, f'{labels[i]}', fontsize=10, ha=alignment, va=vertical_alignment)

    # Hide axes and grid for the n-gon plot
    ax.axis('off')

    # Calculate partition sizes for the rectangle based on the last values plot
    last_values = values_list[-1]
    partition_sizes = [value * 8 for value in last_values]  # Scale values for the rectangle

    # Plot the rectangle with colored partitions and labels
    ax2 = fig.add_axes([1.1, 0.1, 0.15, 0.75])  # Rectangle position and size

    rect_label=copy.deepcopy(labels)

    label_thresh=0.06
    for i in range(len(labels)):
        if last_values[i]<label_thresh:
            rect_label[i]=""

    # colors=colors[::-1]
    # rect_label=rect_label[::-1]
    
    y_position = 0.1  # Initial y-position for the bottom of the partitions
    for i, size in enumerate(partition_sizes):
        ax2.add_patch(plt.Rectangle((0, y_position), 1, size, color=colors[i], alpha=0.7))
        ax2.text(0.5, y_position + size / 2, f'{rect_label[i]}', fontsize=10, ha='center', va='center', color='black')
        y_position += size

    # Set limits and hide axes for the rectangle plot
    ax2.set_xlim(0, 1)
    ax2.set_ylim(0, 8)
    ax2.axis('off')

    # Add a bold title at the top of the figure
    fig.suptitle("Evolution Plot", fontsize=19, fontweight='bold',x=0.75)

    day_text = f"Day - {len(data)}"
    fig.text(0.75, 0.01, day_text, ha='center', fontsize=12)

    # Add table beside the rectangle
    table_data = {
        ' ': labels,
        '': [f'{value:.4f}' for value in last_values]
    }
    df = pd.DataFrame(table_data)
    ax3 = fig.add_axes([1.3, 0.1, 0.23, 0.75], frame_on=False)  # Position and size of the table
    ax3.axis('off')
    table = ax3.table(cellText=df.to_numpy(), colLabels=df.columns, cellLoc='center', loc='center', colColours=['white','white'])
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 1.5)

    for key, cell in table.get_celld().items():
        cell.set_linewidth(0)
        cell.set_text_props(color='black')

        
    # Show the plot
    #plt.show()

    frame_number=len(data)-2

    frame_filename = f"{frames_path}/frame_{frame_number:0{decimal}d}.png"
    plt.savefig(frame_filename, bbox_inches='tight',dpi=200)
    plt.close()


def plot_frames(data,labels,max_plots):

    if(max_plots == None):
        max_plots=len(data)
        
    decimal=len(str(len(data)))+1

    for i in tqdm(range(2,len(data)+1)):
        draw_ngon(data[:i],labels,max_plots,decimal)

# import multiprocessing
# from tqdm import tqdm
# def worker(frame_data, labels, max_plots):
#     draw_ngon(frame_data, labels, max_plots)

# def plot_frames(data, labels, max_plots):
#     if max_plots is None:
#         max_plots = len(data)

#     # Prepare the arguments for each worker
#     tasks = [(data[:i], labels, max_plots) for i in range(2, len(data) + 1)]

#     with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
#         # Use tqdm to show progress
#         for _ in tqdm(pool.starmap(worker, tasks), total=len(tasks)):
#             pass  # Each task is processed in the worker
        
        
def generate_video(size,fps=30):

    def inverse_image(im):
        return (255-im)

    # Get the list of frames
    frame_files = sorted([f for f in os.listdir(frames_path) if f.endswith('.png')])
    print(frame_files)
    frame_files=frame_files[:size-1]
    
    img_path = os.path.join(frames_path, frame_files[-1])
    img = cv2.imread(img_path)
    height, width, _ = img.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # for MP4
    out = cv2.VideoWriter(video_path, fourcc, fps, (width, height))

    # Write all frames to the video
    for file in frame_files:
        img_path = os.path.join(frames_path, file)
        frame = cv2.imread(img_path)
        frame=inverse_image(frame)
        out.write(frame)

    # Release the VideoWriter
    out.release()
    print(f"Video successfully created at {video_path}")



In [4]:
def count_digits_after_decimal(number):
    # Convert the number to a string
    str_number = str(number)
    
    # Find the position of the decimal point
    if '.' in str_number:
        decimal_position = str_number.index('.')
        # Count the number of digits after the decimal point
        return len(str_number) - decimal_position - 1
    else:
        return 0  # No decimal point means no digits after the decimal


In [5]:

def weighted_random(values, weights):
    return random.choices(values, weights=weights, k=1)[0]

In [6]:
def choosen_eater(state):
    gone = weighted_random(list(range(len(state))),state)

    state[gone]-=1

    return state,gone

In [7]:
def mutation_chance(born,mutation,mut_max):

    size=len(born)

    def generate(state):
        pop=copy.deepcopy(state)

        if sum(pop)==0:
            return pop
        
        pos=0

        for i,p in enumerate(pop):
            if p>0:
                pos=i
                break

        for _ in range(pop[pos]):

            rand=random.randint(1,mut_max)

            if rand<=mutation:
                ch=random.randint(1,size-1)
                pop[pos]-=1
                pop[(pos+ch)%size]+=1

        return pop

    rand=[]

    for i,b in enumerate(born):
        r=[0 for _ in range(size)]
        r[i]=b
        nr=generate(r)
        rand.append(nr)

    mut=[sum(values) for values in zip(*rand)]

    return mut

In [8]:

def children_chance(born):
    perfect=int(born)
    chance=born-int(born)

    if chance==0:
        return perfect

    ch_max=10**(count_digits_after_decimal(chance))
    ch=chance*ch_max

    alive=random.randint(1,ch_max)<=ch

    if alive:
        perfect+=1
    return perfect


In [9]:
def random_allele(allele,gene_dominance):
    
    dominance=[gene_dominance[i] for i in allele]
            
    rand = weighted_random(allele,dominance)
    return rand

In [10]:
def simulate_days(gene_dominance=[],allele=None,initial_state=None,days=0,trees=0,mango_matrix=None,mutation=0):

    mut_max=10**(count_digits_after_decimal(mutation))
    mutation*=mut_max


    population=[initial_state]
    #print(population)

    for _ in tqdm(range(days)):
        state=copy.deepcopy(population[-1])
        tree_list=[[0 for _ in range(len(state))] for _ in range(trees)]

        for i in range(len(tree_list)):

            if sum(state) < 2:
                break

            state,eater1=choosen_eater(state)
            state,eater2=choosen_eater(state)
            
            eat1=random_allele(allele[eater1],gene_dominance)
            eat2=random_allele(allele[eater2],gene_dominance)
            #print(population[-1]," ",state," ",eat1," ",eat2)

            tree=[0 for _ in range(len(state))]

            born1=mango_matrix[eat1][eat2] 
            born2=mango_matrix[eat2][eat1]

            born1=children_chance(born1)
            born2=children_chance(born2)

            tree[eater1]+=born1
            tree[eater2]+=born2

            tree=mutation_chance(tree,mutation,mut_max)

            #print(tree)



            tree_list[i]=tree

        new_state=[0 for _ in range(len(state))]
        
        for row in tree_list:
            for i in range(len(state)):
                new_state[i] += row[i]

        population.append(new_state)

    return population



In [11]:

# rr rs rp
# sr ss sp
# pr ps pp

win = 2
draw = 1
loose = 0

# mango_matrix= [
#             [draw, win, loose],
#             [loose, draw, win],
#             [win, loose, draw]
#         ]

def rps_mechanic(n):

    mat=[[0 for _ in range(n)] for _ in range(n)]
    normal=[draw]+[win for _ in range(n//2)]+[loose for _ in range(n//2)]
    
    for i in range(n):
        mat[i]=normal[-i:]+normal[:-i]
    
    return mat

def allele_generator(gene, species, segment):
    unique_lists = set()
    while len(unique_lists) < species:
        # Generate a list of random integers from 0 to n
        new_list = tuple(random.randint(0, gene-1) for _ in range(segment))
        unique_lists.add(new_list)  # Add to set to ensure uniqueness
    
    # Convert each tuple back to a list
    return [list(item) for item in unique_lists]

gene=7
segment=5
species=11

mango_matrix= rps_mechanic(gene)

mutation=0.0002

days=5000

gene_dominance=[1 for _ in range(gene)] #domiance ratio

allele=allele_generator(gene, species, segment)
pprint.pprint(allele)
print("\n\n")
initial_state=[100 for _ in range(species)]

trees=2000

population = simulate_days(gene_dominance=gene_dominance,mango_matrix=mango_matrix,allele=allele,initial_state=initial_state,days=days,trees=trees,mutation=mutation)

print(population[::(len(population)/20)])

[[6, 6, 2, 2, 4],
 [2, 1, 4, 3, 3],
 [0, 4, 2, 1, 1],
 [4, 3, 2, 4, 5],
 [0, 6, 2, 4, 6],
 [4, 1, 1, 6, 5],
 [1, 2, 3, 4, 3],
 [2, 4, 2, 1, 2],
 [6, 2, 6, 2, 0],
 [6, 3, 2, 5, 1],
 [0, 6, 3, 2, 2]]





100%|██████████| 5000/5000 [01:37<00:00, 51.35it/s]


TypeError: slice indices must be integers or None or have an __index__ method

In [ ]:
data=[]

for pop in population:

    tot=sum(pop)

    if tot==0:
        r=1/species
        ratio=[r for _ in range(species)]
    else:
        ratio=[pop[i]/tot for i in range(species)]
        
    data.append(ratio)

print (population[::50])
print(data[-1])
print(len(data))

In [13]:
import shutil
import os

shutil.rmtree(frames_path)

os.makedirs(frames_path)


In [ ]:

hand_signs = ["Rock", "Paper", "Scissors", "Fire", "Water", "Air", "Lightning", "Tree", "Snake", "Human", "Wolf", "Gun", "Dragon", "Sun", "Moon", "Alien", "Magic", "Robot", "Ghost", "Time"]  # The labels for the three variables
alphabet=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']



def concatenate_labels(matrix, strings):
    result = []
    for row in matrix:
        # Use the indices from the row to take elements from the strings list
        concatenated = ','.join(strings[i] for i in row)
        result.append(concatenated)
    return result

labels=concatenate_labels(allele,alphabet)
max_plots=50

plot_frames(data,labels,max_plots)

size=len(data)
print (size)

In [ ]:
generate_video(size,fps=20)